In [1]:
%pip install -q pyomo
!apt-get install -y -qq coinor-cbc

Note: you may need to restart the kernel to use updated packages.
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [2]:
from pyomo.environ import *
import math

n=10
col=range(n)
row=range(n)
model=ConcreteModel()

In [3]:
graph=[[1,2],[1,5],[1,6],[1,8],[2,3],[3,4],[4,10],[4,5],[6,7],[7,8],[7,9]]
for i in range(len(graph)):
  graph[i][0] = graph[i][0] - 1
  graph[i][1] = graph[i][1] - 1


In [4]:
path = [[0 for p in range(n)] for q in range(n)]
for ind in graph:
	path[ind[0]][ind[1]] = 1
	path[ind[1]][ind[0]] = 1
print(path)

[[0, 1, 0, 0, 1, 1, 0, 1, 0, 0], [1, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 1, 0, 0, 0, 0, 1], [1, 0, 0, 1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 1, 1, 0], [1, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]]


In [5]:
model.xij = Var(row, col, domain = Binary)

In [6]:
model.obj = Objective(expr = sum(model.xij[i,j] for i in row for j in col)/2, sense = minimize)

In [7]:
model.constraints = ConstraintList()

for i in row:
  for j in col:
    model.constraints.add(expr = model.xij[i,j] <= path[i][j])
    model.constraints.add(expr = (model.xij[i,j] == model.xij[j,i]))

In [8]:
for i in row:
  model.constraints.add(expr = sum(model.xij[i,j] for j in col)>=1)

In [9]:
model.pprint()

4 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :  210 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 

In [10]:
opt = SolverFactory('cbc')

In [12]:
result = opt.solve(model, tee = True)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Welcome to the CBC MILP Solver 
Version: 2.10.8 
Build Date: May  9 2022 

command line - /bin/cbc -printingOptions all -import /tmp/tmpxzceh9ck.pyomo.lp -stat=1 -solve -solu /tmp/tmpxzceh9ck.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 6 (-204) rows, 9 (-92) columns and 14 (-366) elements
Statistics for presolved model
Original problem has 100 integers (100 of which binary)
Presolved problem has 9 integers (9 of which binary)
==== 0 zero objective 1 different
9 variables have objective of 1
==== absolute objective values 1 different
9 variables have objective of 1
==== for integers 0 zero objective 1 different
9 variables have objective of 1
==== for integers absolute objective values 1 different
9 variables have objective of 1
===== end objective counts


Problem has 6 rows, 9 columns (9 with objective) and 14 elements
There are 4 singletons with objective 
Column breakdown:
0 of type 0.0->inf, 0 of type 0.0->up, 0 of type lo->inf, 
0

Solver status: ok
Solver termination condition: optimal


In [13]:
print('Number of Guards', model.obj())

Number of Guards 6.0
